# Notes 

#### Calculating Passivity Index Through Optimization Methods

In the stability analysis of the networks controlled via some pinning control method, we encounter inequalities fo the form 
$$
    (x - y)^T (f(x) - f(y)) ≤ (x - y)^T K P (x - y) \quad \forall x, y \in \mathbb{R}^d
$$
or of the form 
$$
    (x - y)^T P (f(x) - f(y) -\Delta (x - y)) ≤ -w(x - y)^T (x - y) \quad  w > 0,\; \forall x, y \in \mathbb{R}^d
$$
Once a node dynamics $f$ and inner connectivity matrix $P$ is given, the problem here is to find the matrix positive definite possible diagonal matrix $\Delta$ or $K$. This seems to be solved using semi-definite programming formulated as a feasibility problem that is conditioned on the positive definite matrix $\Delta$ or $K$. *However the main obstacle in using optimization methods to find $K$ or $\Delta$ is that the above inequality cannot be formulated as a linear matrix inequality because of the existence of $f(x)$ and $f(y)$*. 

#### Calculating Control Matrix Using Optimization Methods 

For the full synchronization in undirected networks, we have matrix inequalities of the form 
$$
\Omega = \theta I_n + \epsilon (\Xi - A) < 0 
$$
When a network with fixed topology is given, $\theta$ and $\epsilon$ and $A$ is fixed. Thus, the control matrix $A$ can be solved using semidefinite programming. Here $A$ is in the form 
$$
A = diag(α_1, α_2, \ldots, \alpha_l, 0, \ldots, 0) \in \mathbb{R}^n
$$
Thus the problem is that when additional conditions are imposed in the optimization problem the problem becomes infeasible. 

In [50]:
using JuMP, SCS 
using LightGraphs 
using LinearAlgebra

# Construct a graph 
θ = 10
ϵ = 50.
graph = random_regular_graph(10, 4)
Ξ = -collect(laplacian_matrix(graph))

# Construct an optimization model 
model = Model(SCS.Optimizer)
set_silent(model)

# Add variables 
n = size(Ξ, 1)
@variable(model, A[1 : n, 1 : n], PSD)

# Add constraints 
Γ = θ * I(n) + ϵ * Ξ
@SDconstraint(model, Γ - A ≤ 0)

# Optimize the model 
optimize!(model)

# Get the value 
Aval = value.(A)

# Checks 
@show all(eigvals(Aval) .> 0)
@show all(eigvals(Γ - Aval) .< 0);
display(Aval)

10×10 Matrix{Float64}:
  474.701       5.93658     5.93658  …  -117.423       5.93658     5.93658
    5.93658   474.701       5.93658        5.93658  -117.423       5.93658
    5.93658     5.93658   474.701          5.93658  -117.423    -117.423
 -117.423    -117.423       5.93658     -117.423       5.93658     5.93658
 -117.423    -117.423    -117.423       -117.423       5.93658     5.93658
    5.93658  -117.423       5.93658  …  -117.423    -117.423    -117.423
 -117.423       5.93658  -117.423          5.93658     5.93658  -117.423
 -117.423       5.93658     5.93658      474.701       5.93658     5.93658
    5.93658  -117.423    -117.423          5.93658   474.701    -117.423
    5.93658     5.93658  -117.423          5.93658  -117.423     474.701

all(eigvals(Aval) .> 0) = true
all(eigvals(Γ - Aval) .< 0) = true


In the above model, $A$ is positive definite but is not a diagonal matrix. For this we can impose additional constraints the optimization model.

In [51]:
# Construct an optimization model 
model = Model(SCS.Optimizer)
set_silent(model)

# Add variables 
n = size(Ξ, 1)
@variable(model, A[1 : n, 1 : n], PSD)

# Add constraints 
Γ = θ * I(n) + ϵ * Ξ
@SDconstraint(model, Γ - A ≤ 0)
for i in 1 : n 
    for j in 1 : n 
        if i ≠ j 
            @constraint(model, A[i, j] == 0) # Off diagonal elements are zero
        end 
    end 
end 

# Optimize the model 
optimize!(model)

# Get the value 
Aval = value.(A)

# Checks 
@show all(eigvals(Aval) .> 0)
@show all(eigvals(Γ - Aval) .< 0);
display(Aval)

10×10 Matrix{Float64}:
 92.9192      -6.76988e-7  -5.99039e-7  …  -5.29143e-7  -5.4324e-7
 -6.76988e-7  89.9483      -7.11533e-7      4.75597e-7  -7.1869e-7
 -5.99039e-7  -7.11533e-7  90.9425          3.97649e-7   4.05936e-7
  3.59972e-7   5.14975e-7  -6.17981e-7     -5.61363e-7  -5.72782e-7
  4.15772e-7   5.07817e-7   5.45312e-7     -6.17162e-7  -6.23943e-7
 -6.01364e-7   4.84861e-7  -7.5599e-7   …   3.99973e-7   4.08067e-7
  4.73896e-7  -6.95734e-7   5.11142e-7     -6.75287e-7   5.18859e-7
  3.65429e-7  -6.81821e-7  -6.21188e-7     -5.66819e-7  -5.77785e-7
 -5.29143e-7   4.75597e-7   3.97649e-7     92.9192       3.41849e-7
 -5.4324e-7   -7.1869e-7    4.05936e-7      3.41849e-7  92.4379

all(eigvals(Aval) .> 0) = true
all(eigvals(Γ - Aval) .< 0) = true


However, we a need a matrix of the form $A = diag(\alpha_1, \alpha_2, \ldots, \alpha_l, 0, \ldots, 0)$, where $l$ is the number of pinned nodes. So we need additional constraints for the diagonal elements $A[i, i] = 0, i = l + 1, \ldots, n, j = l + 1, \ldots, n$. 

In [52]:
# Construct an optimization model 
model = Model(SCS.Optimizer)
set_silent(model)

# Add variables 
n = size(Ξ, 1)
@variable(model, A[1 : n, 1 : n], PSD)

l = n ÷ 2

# Add constraints 
Γ = θ * I(n) + ϵ * Ξ
@SDconstraint(model, A ≥ 0 )
@SDconstraint(model, Γ - A ≤ 0)
for i in 1 : n 
    for j in 1 : n 
        if i ≠ j 
            @constraint(model, A[i, j] == 0) # Off diagonal elements are zero
        end 
    end 
end 
for i in l + 1 : n
    @constraint(model, A[i, i] == 0) # Diagonal element constraint 
end 

# Optimize the model 
optimize!(model)

# Get the value 
Aval = value.(A)

# Checks 
@show termination_status(model)
@show all(eigvals(Aval) .> 0)
@show all(eigvals(Γ - Aval) .< 0);
display(Aval)

10×10 Matrix{Float64}:
 25.6232        0.000489618   0.000193532  …   9.68632e-5   -1.13862e-5
  0.000489618  24.9779        0.000356973     -0.000200458  -0.000102974
  0.000193532   0.000356973  25.5717          -0.000136879  -0.000296567
 -0.000273207   0.000119293   0.000201699      6.39848e-5   -4.29235e-5
 -0.000107916   0.00015137    5.89899e-6       3.44406e-5    9.18448e-5
 -2.45657e-5   -0.000232751  -5.32439e-5   …  -1.18755e-5    0.000107337
 -7.67006e-6    0.000174684  -0.000169886     -0.000108533   5.9469e-6
 -4.00991e-6   -4.56302e-5    0.000226638      2.83474e-5   -8.61897e-6
  9.68632e-5   -0.000200458  -0.000136879     -8.39638e-6    4.71954e-6
 -1.13862e-5   -0.000102974  -0.000296567      4.71954e-6    0.000104955

termination_status(model) = MathOptInterface.OPTIMAL
all(eigvals(Aval) .> 0) = false
all(eigvals(Γ - Aval) .< 0) = true


Note that the optimization problem is terminated with a matrix $A$ whose eigenvalues are not greated that zero.

#### Irrducible Initial Matrix For Cluster Synchronization 

For the cluster synchronization of cluster synchroniztion we need an initial matrix of the form 
$$
\Phi = \begin{bmatrix}
\Phi_{11} & \Phi_{12} & \ldots & \Phi_{1l} \\ 
\Phi_{21} & \Phi_{22} & \ldots & \Phi_{2l} \\ 
\vdots & \vdots & \ddots & \vdots \\ 
\Phi_{l1} & \Phi_{l2} & \ldots & \Phi_{ll} \\ 
\end{bmatrix}
$$
The condition on $\Phi$ is that all the blocks must be zero sum and additiionally all the blocks on the diagonal must be irreducible. Thus we need a method for initial construction of the matrix $\Phi$ that satisfy the conditions on the block matrices. The matrix $\Phi$ must also be symmetric.

The possible approach would like this: If a matrix $A$ is irreducile, then it is associated graph is strongly connected. Thus, we can construct initially some strongly connected subgraphs and then we can merge them together to construct the matrix $\Phi$ representing the whole graph. 

The main diffuculty here is that satisfying the zero row sum and while keeping the subblock matrices symmetric may not be possible all the time. 